- grid2region:栅格转化为区域标签{"2065-1268": 5761}
- region_distance:区域之间距离

In [1]:
import json
import numpy as np

grid2region = json.load(open('../data/region_data/region_beijing_grid2region_REID_no_less10.json'))  #GRID2REGION lookup
region_distacne = np.load('../data/region_data/region_distance_no_less10.npy') #Reigon distance lookup
print('Shape of region distance:', region_distacne.shape)

Shape of region distance: (10656, 10656)


区域边界

In [2]:
RN = len(region_distacne)
lng_ld = 116.1
lat_ld = 39.7
lng_max = 116.7
lat_max = 40.2
horizontal_number = 3000
vertical_number = 2500

坐标转化为栅格，栅格转化为区域标签

In [3]:
def transcoor2grid(lng,lat):
	return str(int((lng-lng_ld)/0.0002)) + '-' + str(int((lat-lat_ld)/0.0002))

def transgrid2region(grid):
	count = 0
	while(1):
		if grid in grid2region:
			return grid2region[grid]
		else:
			grid_lng, grid_lat = int(grid.split('-')[0]),int(grid.split('-')[1])
			if count % 2 == 0:
				if grid_lat >= 1:
					grid_lat -= 1
			else: 
				if grid_lng >= 1:
					grid_lng -= 1
			grid = str(grid_lng) + '-' + str(grid_lat)
			count += 1

print(transcoor2grid(116.2356,39.8423))   
print(transgrid2region("2065-1268"))        

678-711
5761


加载参数

In [4]:
import tensorflow as tf

/home/liwei/anaconda3/envs/attnmove/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/liwei/anaconda3/envs/attnmove/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/liwei/anaconda3/envs/attnmove/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/liwei/anaconda3/envs/attnmove/lib/pyt

In [5]:
import self_attn_hyperparams_region
# jupyter中使用hyperparams需要添加
import sys
sys.argv = ['run.py']

hparams = self_attn_hyperparams_region.load_hyperparams()
train_dataset_hparams, valid_dataset_hparams, test_dataset_hparams, \
	decoder_hparams, opt_hparams, opt_vars, loss_hparams, args = \
		hparams['train_dataset_hparams'], hparams['eval_dataset_hparams'], \
		hparams['test_dataset_hparams'], hparams['decoder_hparams'], \
		hparams['opt_hparams'], hparams['opt_vars'], \
		hparams['loss_hparams'], hparams['args']

max_seq_length =  288
train_file:/home/liwei/AttnMove/data/pos.train.txt
valid_file:/home/liwei/AttnMove/data/pos.validate.txt
test_file:/home/liwei/AttnMove/data/pos.test.txt
vocab_file:/home/liwei/AttnMove/data/pos.vocab.txt
logdir:../log/log_dir/pos.bsize50.epoch500.seqlen288.dynamic_lr.present0.75.partition10.hidden64.self_attn/


In [6]:
import os
import setproctitle
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
setproctitle.setproctitle(args.process_name)

周末为0，weekend为1

In [7]:
# Dataset 
day_dic = {"0":1, "1":0, "2":0,
			"3":1, "4":1, "5":1, "6":1, "7":1, "8":0, "9":0, 
			"10":1, "11":1, "12":1, "13":1, "14":1, "15":0, "16":0,
			"17":0, "18":1,"19":1, "20":1, "21":1, "22":0, "23":0,
			"24":1, "25":1, "26":1, "27":1, "28":1, "29":0}
print(train_dataset_hparams)
print(valid_dataset_hparams)
print(test_dataset_hparams)


{'num_epochs': 1, 'seed': 1234, 'shuffle': False, 'dataset': {'files': '/home/liwei/AttnMove/data/pos.train.txt', 'vocab_file': '/home/liwei/AttnMove/data/pos.vocab.txt', 'max_seq_length': 288, 'bos_token': '<BOS>', 'eos_token': '<EOS>', 'length_filter_mode': 'truncate'}, 'batch_size': 50, 'allow_smaller_final_batch': False}
{'num_epochs': 1, 'seed': 1234, 'shuffle': False, 'dataset': {'files': '/home/liwei/AttnMove/data/pos.validate.txt', 'vocab_file': '/home/liwei/AttnMove/data/pos.vocab.txt', 'max_seq_length': 288, 'bos_token': '<BOS>', 'eos_token': '<EOS>', 'length_filter_mode': 'truncate'}, 'batch_size': 50, 'allow_smaller_final_batch': False}
{'num_epochs': 1, 'seed': 1234, 'shuffle': False, 'dataset': {'files': '/home/liwei/AttnMove/data/pos.test.txt', 'vocab_file': '/home/liwei/AttnMove/data/pos.vocab.txt', 'max_seq_length': 288, 'bos_token': '<BOS>', 'eos_token': '<EOS>', 'length_filter_mode': 'truncate'}, 'batch_size': 50, 'allow_smaller_final_batch': False}


In [8]:
import dataset_utils
from dataset_utils import TrainTestDataIterator

word2id, id2word, traindata, testdata, trainuser, traintime, testuser, testtime, validdata, validuser, validtime, trainday, testday, validday = \
				dataset_utils.loadDataset(train_dataset_hparams, valid_dataset_hparams, test_dataset_hparams, day_dic)

Loading training dataset from /home/liwei/AttnMove/data/pos.train.txt
Loading testing dataset from /home/liwei/AttnMove/data/pos.test.txt
Loading valid dataset from /home/liwei/AttnMove/data/pos.validate.txt
Loading vocabulary dataset from /home/liwei/AttnMove/data/pos.vocab.txt


In [ ]:
print(word2id)
print(id2word)
